In [8]:
#!pip3 install git+https://github.com/aatimofeev/spacy_russian_tokenizer.git
from spacy.lang.ru import Russian
from tqdm import tqdm_notebook as tqdm

In [9]:
MAKE_CHUNK = 1
COUNT_CHUNKS = 2
WIKI_SIZE = 967783 # make smaller, due to limits of machine 9677833

In [10]:

block_sz = WIKI_SIZE / COUNT_CHUNKS
start = int(MAKE_CHUNK * block_sz)
end = int(start + block_sz)
ps = []
line_i = 0
with open('service/data/wiki/wiki_big.txt') as f:
    for i, line in enumerate(f):
        if line_i >= end:
                break 
        for x in line.split('.'):
            if line_i >= start:
                ps += [x]
            line_i += 1
            if line_i >= end:
                break

In [11]:
import gc
gc.collect()

64

In [12]:
len(ps)

483891

In [13]:
import gensim
from gensim.test.utils import datapath
from collections import defaultdict
wv_from_bin = gensim.models.Word2Vec.load("ru.bin")


docs_by_tokens = defaultdict(lambda : defaultdict(int))
wv_by_token = {}

nlp = Russian()



for i, p in enumerate(tqdm(ps)):
    p = p.lower()
    doc = nlp(p)
    tokens = [token.text for token in doc]    
    for it in range(len(tokens)):
        docs_by_tokens[tokens[it]][i] += 1
        if it > 0:
            docs_by_tokens[' '.join([tokens[it-1], tokens[it]])][i] += 1

In [14]:
import json

with open('wiki_texts{}.txt'.format(MAKE_CHUNK), 'w') as f:
    json.dump(ps, f)
    
with open('wiki_doc_by_tokens{}.txt'.format(MAKE_CHUNK), 'w') as f:
    json.dump(docs_by_tokens, f)